In [1]:
%load_ext autoreload  
%autoreload 2 
import datasets
from subset_active_learning.subset_selection import select, preprocess, compare, correlation
import wandb
import numpy as np
import torch

/home/glai/anaconda3/envs/subset/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PROXY_MODEL_NAME = "google/electra-small-discriminator"
PROXY_TRAINING_ARGS = select.SubsetTrainingArguments(model_card=PROXY_MODEL_NAME)
TARGET_MODEL_NAME = "roberta-base"
TARGET_TRAINING_ARGS = select.SubsetTrainingArguments(model_card=TARGET_MODEL_NAME) 
WANDB_TAGS = ["correlation", "fixed-roberta"]

correlation_run = correlation.CorrelationRun(
    proxy_model_name=PROXY_MODEL_NAME,
    target_model_name=TARGET_MODEL_NAME,
    proxy_training_config=PROXY_TRAINING_ARGS,
    target_training_config=TARGET_TRAINING_ARGS,
    wandb_tags = WANDB_TAGS
)

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 725.95it/s]
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-7b23d5d1250b3cef.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6a94e7b0eb5aeeeb.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-f1806eefe95f0de4.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-e283858a38c31b48.arrow
Loading cache

In [8]:
correlation_run.wandb_tags

['correlation', 'fixed-roberta']

In [9]:
correlation_run.one_comparison_run(
    data_seed=42, 
    model_seed=1,
    subset_size=500
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: garylai (johnny-gary). Use `wandb login --relogin` to force relogin


##### Warning: Hard Setting Model Seed to 1


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
for TRAIN_SIZE in (500,):
    for SEED in range(42, 62):
        comparison_run = compare.ComparisonRun(train_ds = random_select_subset(ds=processed_ds["train"], seed=SEED, size=TRAIN_SIZE), \
                                                valid_ds = processed_ds["validation"], \
                                                test_ds = processed_ds["test"], \
                                                seed = SEED)
        for MODEL_CARD in ("roberta-base", "google/electra-small-discriminator"):
            comparison_run.one_run(wandb_tags=[MODEL_CARD, "correlation", f"train_size-{TRAIN_SIZE}", "new_param"], config=select.SubsetTrainingArguments(model_card=MODEL_CARD))